# Clean raw Bigfood data

***Objective:*** Take the raw data and clean it up a little bit to get it ready for next steps.

## Required libraries

In [1]:
from datetime import datetime
import os
import pandas as pd

## Required function

In [2]:
def zodiac_sign(day, month):
    # Adapted from:
    #     https://stackoverflow.com/questions/61537916/map-column-birthdates-in-python-pandas-df-to-astrology-signs
    #
    # PURPOSE
    #     This function will receive a day and month as arguments and 
    #     then return the zodiac sign for the date.
    #
    # INPUTS
    #     day   := The day of the month. Type should be numeric.
    #     month := The full name of the month. Type should be a string and all 
    #              chars should be lowercase.
    #
    # OUTPUT
    #     N/A.
    #
    # RETURNS
    #     The zodiac sign of the date provided. Type will be a string.
    if month == 'december': 
        return 'Sagittarius' if (day < 22) else 'Capricorn'
    elif month == 'january': 
        return 'Capricorn' if (day < 20) else 'Aquarius'
    elif month == 'february': 
        return 'Aquarius' if (day < 19) else 'Pisces'
    elif month == 'march': 
        return 'Pisces' if (day < 21) else 'Aries'
    elif month == 'april': 
        return 'Aries' if (day < 20) else 'Taurus'
    elif month == 'may': 
        return 'Taurus' if (day < 21) else 'Gemini'
    elif month == 'june': 
        return 'Gemini' if (day < 21) else 'Cancer'
    elif month == 'july': 
        return 'Cancer' if (day < 23) else 'Leo'
    elif month == 'august': 
        return 'Leo' if (day < 23) else 'Virgo'
    elif month == 'september': 
        return 'Virgo' if (day < 23) else 'Libra'
    elif month == 'october': 
        return 'Libra' if (day < 23) else 'Scorpio'
    elif month == 'november': 
        return 'scorpio' if (day < 22) else 'Sagittarius'

## Bring the data in

In [3]:
os.chdir("../data/raw/")

In [4]:
dat = pd.read_csv("bfro_reports_geocoded.csv")

In [5]:
# Remove the observations not having a lat/long because I want to plot the
# locations of the sightings
dat = dat[dat["longitude"].notna()]

In [6]:
# Remove the observations not having a high or low temp recorded
dat = dat[dat["temperature_high"].notna()]

In [7]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4102 entries, 1 to 5073
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   observed            4069 non-null   object 
 1   location_details    3493 non-null   object 
 2   county              4102 non-null   object 
 3   state               4102 non-null   object 
 4   season              4102 non-null   object 
 5   title               4102 non-null   object 
 6   latitude            4102 non-null   float64
 7   longitude           4102 non-null   float64
 8   date                4102 non-null   object 
 9   number              4102 non-null   float64
 10  classification      4102 non-null   object 
 11  geohash             4102 non-null   object 
 12  temperature_high    4102 non-null   float64
 13  temperature_mid     3964 non-null   float64
 14  temperature_low     4102 non-null   float64
 15  dew_point           3951 non-null   float64
 16  humidi

## Drop unnecessary columns

In [8]:
dat = dat.drop(["location_details",
                "title",
                "number",
                "geohash",
                "precip_type",
                "summary",
                "uv_index"], axis=1)

## Get rid of the Class C reports

The BFRO defines Class C reports as follows: 

>Most second-hand reports, and any third-hand reports, or stories with an untraceable sources, are considered Class C, because of the high potential for inaccuracy. Those reports are kept in BFRO archives but are very rarely listed publicly in this database. The exceptions are for published, or locally documented incidents from before 1958 (before the word "Bigfoot" entered the American vocabulary), and sightings mentioned in non-tabloid newspapers or magazines. https://www.bfro.net/GDB/classify.asp

Given the above definition we will remove them.

In [9]:
dat = dat[dat["classification"] != "Class C"]

## Create weather related features

### Snow or rain feature

In [10]:
rain_snow = []
for condition in dat["conditions"]:
    if type(condition) == float: # Deal with the NaNs
        rain_snow.append("Unknown")
    else:
        if "rain" in condition.lower() and "snow" in condition.lower():
            rain_snow.append("Rain & Snow")
        elif "rain" in condition.lower():
            rain_snow.append("Rain")
        elif "snow" in condition.lower():
            rain_snow.append("Snow")
        else:
            rain_snow.append("No Rain or Snow")
dat["rain_or_snow"] = rain_snow
del rain_snow

### Cloud cover feature

In [11]:
cloud_conditions = []
for condition in dat["conditions"]:
    if type(condition) == float:
        cloud_conditions.append("Unknown")
    else:
        if "overcast" in condition.lower():
            cloud_conditions.append("Overcast")
        elif "partially" in condition.lower():
            cloud_conditions.append("Partially Cloudy")
        else:
            cloud_conditions.append("Clear")
del condition
dat["cloudy_or_overcast"] = cloud_conditions
del cloud_conditions
dat = dat.drop(["conditions"], axis=1)

### Temperature range

In [12]:
dat["temperature_range"] = dat["temperature_high"] - dat["temperature_low"]

## Code a "better" `moon_phase` feature

While the raw data has a feature for the moon phase at the time of the sighting (`moon_phase`), it is a float in $[0,1)$ and not easily human interpretable. To make the moon phase more easier to understand, we will code a categorical feature for the moon phase called `moon_phase_cycle`.

The mapping of the original values into string is taken from the weather site (Visual Crossing) that the original weather data came from. Below is the breakdown that they provide.

>The moon phase number represents the fractional portion through the current moon lunation cycle ranging from 0 (the new moon) to 0.5 (the full moon) and back to 1 (the next new moon). These numbers are similar to other Weather API providers such as Dark Sky so converting existing applications is easy.
>Traditional, western moon cycles are therefore represented by the following values:
>
>* 0 – new moon
>* 0-0.25 – waxing crescent
>* 0.25 – first quarter
>* 0.25-0.5 – waxing gibbous
>* 0.5 – full moon
>* 0.5-0.75 – waning gibbous
>* 0.75 – last quarter
>* 0.75 -1 – waning crescent
>
>Moon phases are reported at midday local time. On days where a quarterly phase occurs (new moon, first quarter, full moon and last quarter) the values are rounded to the quarterly phase value (0, 0.25, 0.5, 0.75). Therefore you can identify the quarterly days by identifying days with these values.
>
>Source: https://www.visualcrossing.com/resources/documentation/weather-api/how-to-include-sunrise-sunset-and-moon-phase-data-into-your-api-requests/


In [13]:
moon_phase_cycle = []
for phase in dat["moon_phase"]:
    if phase == 0:
        moon_phase_cycle.append("New Moon")
    elif phase > 0 and phase < 0.25:
        moon_phase_cycle.append("Waxing Crescent")
    elif phase == 0.25:
        moon_phase_cycle.append("First Quarter")
    elif phase > 0.25 and phase < 0.5:
        moon_phase_cycle.append("Waxing Gibbous")
    elif phase == 0.5:
        moon_phase_cycle.append("Full Moon")
    elif phase > 0.5 and phase < 0.75:
        moon_phase_cycle.append("Waning Gibbous")
    elif phase == 0.75:
        moon_phase_cycle.append("Last Quarter")
    elif phase > 0.75 and phase < 1:
        moon_phase_cycle.append("Waning Crescent")
    else: # Likely not needed, will act as a canary for EDA
        moon_phase_cycle.append("INVALID PHASE")
del phase
dat = dat.drop(["moon_phase"], axis=1)
dat["moon_phase_cycle"] = moon_phase_cycle
del moon_phase_cycle

## Create features based on the date of the sighting

### Day of the week

In [14]:
dat["day_of_week"] = pd.to_datetime(dat["date"]).dt.day_name()

### Create a flag for weekend

In [15]:
dat["weekend_flag"] = ["Yes" if day in ["Saturday", "Sunday"] else "No" 
                       for day in dat["day_of_week"]]

### Zodiac sign on the day of the sighting

In [16]:
dat["temp_date"] = pd.to_datetime(dat["date"])
dat["zodiac_sign_of_sighting_date"] = dat["temp_date"].apply(lambda x: zodiac_sign(x.day, x.strftime("%B").lower()))
dat = dat.drop(["temp_date"], axis=1)

## Write to file

In [17]:
os.chdir("../interim/")

In [18]:
file_name_root = "add_init_raw_feats_"
date_time_str = datetime.today().strftime("%Y-%m-%d %H:%M")
date_str = date_time_str.split()[0].replace("-", "") + "_"
time_str = date_time_str.split()[1].replace(":", "")
del date_time_str
file_ext = ".csv"
file_name = file_name_root + date_str + time_str + file_ext
del file_name_root
del date_str
del time_str
del file_ext

In [19]:
print(file_name)

add_init_raw_feats_20230409_1433.csv


In [20]:
dat.to_csv(file_name, sep=",", index=False)
del file_name

In [21]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4086 entries, 1 to 5073
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   observed                      4053 non-null   object 
 1   county                        4086 non-null   object 
 2   state                         4086 non-null   object 
 3   season                        4086 non-null   object 
 4   latitude                      4086 non-null   float64
 5   longitude                     4086 non-null   float64
 6   date                          4086 non-null   object 
 7   classification                4086 non-null   object 
 8   temperature_high              4086 non-null   float64
 9   temperature_mid               3949 non-null   float64
 10  temperature_low               4086 non-null   float64
 11  dew_point                     3936 non-null   float64
 12  humidity                      3936 non-null   float64
 13  clo